# Insurance Conversion Propensity Modeling (Colab)

This notebook is designed to run end-to-end in Google Colab for ranking users by conversion propensity (`has_sale`) with **Average Precision (PR-AUC)** as the primary model selection metric.

## 0 — Setup

In [ ]:
# Colab setup (safe to run in local too)
!pip -q install lightgbm imbalanced-learn optuna optuna-integration shap joblib

import os
import random
import warnings
from dataclasses import dataclass

import joblib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import shap
from imblearn.ensemble import BalancedRandomForestClassifier
from lightgbm import LGBMClassifier
from optuna.integration import lightgbm as optuna_lgb
from scipy import sparse
from sklearn.base import clone
from sklearn.calibration import calibration_curve
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    average_precision_score,
    brier_score_loss,
    precision_recall_curve,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

warnings.filterwarnings('ignore')
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

plt.style.use('seaborn-v0_8-whitegrid')

## 1 — Load + sanity checks

In [ ]:
DATA_PATH = 'data_(2).csv'
df = pd.read_csv(DATA_PATH)
df['dt'] = pd.to_datetime(df['dt'])

print('Shape:', df.shape)
print('
Dtypes:
', df.dtypes)
missing_tbl = df.isna().mean().sort_values(ascending=False).rename('missing_rate')
print('
Missingness:
', (missing_tbl * 100).round(2).astype(str) + '%')

before_n = len(df)
before_pos = df['has_sale'].sum()

dup_mask = df.duplicated(keep='first')
num_dups = int(dup_mask.sum())
df = df.loc[~dup_mask].copy()

after_n = len(df)
after_pos = df['has_sale'].sum()

print(f"\nRemoved duplicates: {num_dups}")
print(f"Rows: {before_n} -> {after_n}")
print(f"Positive count: {before_pos} -> {after_pos}")
print(f"Positive rate: {before_pos / before_n:.4f} -> {after_pos / after_n:.4f}")

assert num_dups == 164, f'Expected 164 duplicates from data facts, got {num_dups}'

uniq_comm = df['commission_rate'].nunique(dropna=False)
print('commission_rate unique values:', uniq_comm)
print('commission_rate values:', df['commission_rate'].dropna().unique()[:5])
assert uniq_comm == 1, 'commission_rate should be constant per provided data facts.'

df = df.drop(columns=['commission_rate'])
print('Dropped commission_rate. New shape:', df.shape)

## 2 — EDA with figures

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Target count and rate
counts = df['has_sale'].value_counts().sort_index()
axes[0].bar(['No Sale (0)', 'Sale (1)'], counts.values, color=['#4C72B0', '#55A868'])
axes[0].set_title('Target Counts')
axes[0].set_ylabel('Count')

rate = df['has_sale'].mean()
axes[1].bar(['Conversion Rate'], [rate], color='#55A868')
axes[1].set_ylim(0, 1)
axes[1].set_title(f'Conversion Rate: {rate:.2%}')
axes[1].set_ylabel('Rate')

plt.tight_layout()
plt.show()

**Insight:** Conversion is imbalanced (~22% positive), so model comparison should prioritize ranking quality metrics like PR-AUC rather than accuracy.

In [ ]:
missing = df.isna().mean().sort_values(ascending=False)
missing = missing[missing > 0]

plt.figure(figsize=(8, 4))
plt.bar(missing.index, missing.values * 100, color='#C44E52')
plt.title('Missingness by Column (%)')
plt.ylabel('Missing %')
plt.xticks(rotation=30, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
platform_plot = df.copy()
platform_plot['platform'] = platform_plot['platform'].fillna('Missing')
platform_rate = platform_plot.groupby('platform')['has_sale'].mean().sort_values(ascending=False)

plt.figure(figsize=(6, 4))
plt.bar(platform_rate.index, platform_rate.values, color='#8172B2')
plt.title('Conversion Rate by Platform')
plt.ylabel('Conversion Rate')
plt.ylim(0, max(platform_rate.values)*1.2)
plt.tight_layout()
plt.show()

In [ ]:
member_plot = df.copy()
member_plot['membership_level'] = member_plot['membership_level'].fillna('Missing')
member_rate = member_plot.groupby('membership_level')['has_sale'].mean().sort_values(ascending=False)

plt.figure(figsize=(7, 4))
plt.bar(member_rate.index, member_rate.values, color='#64B5CD')
plt.title('Conversion Rate by Membership Level (with Missing)')
plt.ylabel('Conversion Rate')
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()

In [ ]:
num_cols = ['age', 'monthly_cost', 'session_time', 'household_income']
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
axes = axes.flatten()

for ax, col in zip(axes, num_cols):
    ax.hist(df.loc[df['has_sale'] == 0, col], bins=30, alpha=0.55, label='No Sale', density=True)
    ax.hist(df.loc[df['has_sale'] == 1, col], bins=30, alpha=0.55, label='Sale', density=True)
    ax.set_title(f'{col} distribution by target')
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Quantile-binned conversion rate vs numeric features
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
axes = axes.flatten()

for ax, col in zip(axes, num_cols):
    tmp = df[[col, 'has_sale']].copy()
    tmp['bin'] = pd.qcut(tmp[col], q=10, duplicates='drop')
    agg = tmp.groupby('bin')['has_sale'].mean().reset_index()
    ax.plot(range(len(agg)), agg['has_sale'], marker='o')
    ax.set_title(f'Binned conversion rate vs {col}')
    ax.set_xlabel('Quantile bin index')
    ax.set_ylabel('Conversion rate')

plt.tight_layout()
plt.show()

**Insights:** `platform` is a strong separator; mid-range age appears to convert more than extremes; lower `monthly_cost` segments appear to convert better. These are ranking signals to capture in models.

## 3 — Feature engineering + preprocessing (sklearn Pipeline)

In [ ]:
EPS = 1e-6

def add_features(frame: pd.DataFrame) -> pd.DataFrame:
    out = frame.copy()
    out['day_of_week'] = out['dt'].dt.dayofweek
    out['is_weekend'] = (out['day_of_week'] >= 5).astype(int)
    out['day_of_month'] = out['dt'].dt.day
    out['month'] = out['dt'].dt.month
    out['cost_income_ratio'] = out['monthly_cost'] / np.maximum(out['household_income'].values, EPS)
    return out

work_df = add_features(df)

TARGET = 'has_sale'
ID_COL = 'id'
DROP_COLS = [TARGET, ID_COL, 'dt']

X = work_df.drop(columns=DROP_COLS)
y = work_df[TARGET].astype(int)

num_features = X.select_dtypes(include=['number']).columns.tolist()
cat_features = X.select_dtypes(exclude=['number']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler()),
        ]), num_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('ohe', OneHotEncoder(handle_unknown='ignore')),
        ]), cat_features),
    ]
)

print('Numeric features:', num_features)
print('Categorical features:', cat_features)

## 4 — Modeling & evaluation (ranking-focused)

In [ ]:
@dataclass
class EvalResult:
    model_name: str
    pr_auc: float
    roc_auc: float


def precision_recall_at_k(y_true, y_score, k_frac):
    n = len(y_true)
    k = max(1, int(np.floor(n * k_frac)))
    order = np.argsort(-y_score)
    top_idx = order[:k]
    y_top = np.asarray(y_true)[top_idx]
    precision_k = y_top.mean()
    recall_k = y_top.sum() / np.asarray(y_true).sum()
    threshold = np.sort(y_score)[-k]
    return {'k_frac': k_frac, 'k': k, 'threshold': float(threshold), 'precision@k': float(precision_k), 'recall@k': float(recall_k)}


def evaluate_scores(y_true, y_score):
    return average_precision_score(y_true, y_score), roc_auc_score(y_true, y_score)

# Stratified split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

models = {
    'LogisticRegression': Pipeline([
        ('prep', preprocessor),
        ('model', LogisticRegression(max_iter=3000, class_weight='balanced', random_state=SEED))
    ]),
    'BalancedRandomForest': Pipeline([
        ('prep', preprocessor),
        ('model', BalancedRandomForestClassifier(
            n_estimators=400, min_samples_leaf=2, random_state=SEED, n_jobs=-1
        ))
    ]),
    'LGBM_baseline': Pipeline([
        ('prep', preprocessor),
        ('model', LGBMClassifier(
            n_estimators=400, learning_rate=0.05, num_leaves=31,
            class_weight='balanced', random_state=SEED, n_jobs=-1, verbosity=-1
        ))
    ]),
}

results = []
test_scores = {}
for name, pipe in models.items():
    pipe.fit(X_train, y_train)
    proba = pipe.predict_proba(X_test)[:, 1]
    pr, roc = evaluate_scores(y_test, proba)
    results.append(EvalResult(name, pr, roc))
    test_scores[name] = proba

res_df = pd.DataFrame([r.__dict__ for r in results]).sort_values('pr_auc', ascending=False)
print(res_df)

In [ ]:
# Alternative time-based split (earlier -> later)
time_sorted = work_df.sort_values('dt').reset_index(drop=True)
cut = int(len(time_sorted) * 0.8)
train_time = time_sorted.iloc[:cut]
test_time = time_sorted.iloc[cut:]

X_train_t = train_time.drop(columns=DROP_COLS)
y_train_t = train_time[TARGET].astype(int)
X_test_t = test_time.drop(columns=DROP_COLS)
y_test_t = test_time[TARGET].astype(int)

time_compare = []
for name, pipe in models.items():
    pipe_t = clone(pipe)
    pipe_t.fit(X_train_t, y_train_t)
    p = pipe_t.predict_proba(X_test_t)[:, 1]
    pr, roc = evaluate_scores(y_test_t, p)
    time_compare.append({'model': name, 'time_split_pr_auc': pr, 'time_split_roc_auc': roc})

print(pd.DataFrame(time_compare).sort_values('time_split_pr_auc', ascending=False))

In [ ]:
# Curves on stratified test split for baseline models
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
for name, score in test_scores.items():
    precision, recall, _ = precision_recall_curve(y_test, score)
    plt.plot(recall, precision, label=f"{name} (AP={average_precision_score(y_test, score):.3f})")
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves')
plt.legend()

plt.subplot(1,2,2)
for name, score in test_scores.items():
    fpr, tpr, _ = roc_curve(y_test, score)
    plt.plot(fpr, tpr, label=f"{name} (ROC-AUC={roc_auc_score(y_test, score):.3f})")
plt.plot([0,1],[0,1],'k--',alpha=0.5)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC Curves')
plt.legend()
plt.tight_layout()
plt.show()

## 5 — Hyperparameter tuning (optimize PR-AUC)

### 5A) Balanced Random Forest tuning (RandomizedSearchCV on Average Precision)

In [ ]:
brf_pipe = Pipeline([
    ('prep', preprocessor),
    ('model', BalancedRandomForestClassifier(random_state=SEED, n_jobs=-1))
])

param_dist = {
    'model__n_estimators': [200, 300, 400, 500, 700],
    'model__max_depth': [None, 4, 6, 8, 12, 16],
    'model__min_samples_leaf': [1, 2, 4, 8],
    'model__max_features': ['sqrt', 'log2', 0.5, 0.8],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

brf_search = RandomizedSearchCV(
    brf_pipe,
    param_distributions=param_dist,
    n_iter=30,
    scoring='average_precision',
    cv=cv,
    random_state=SEED,
    n_jobs=-1,
    verbose=1,
)
brf_search.fit(X_train, y_train)

best_brf = brf_search.best_estimator_
brf_test_proba = best_brf.predict_proba(X_test)[:, 1]
print('Best BRF params:', brf_search.best_params_)
print('Best BRF CV AP:', brf_search.best_score_)
print('Best BRF test AP:', average_precision_score(y_test, brf_test_proba))
print('Best BRF test ROC-AUC:', roc_auc_score(y_test, brf_test_proba))

### 5B) LightGBM tuning with Optuna integration (LightGBMTunerCV)

In [ ]:
# Prepare transformed matrices for LightGBM tuner
prep_no_scale = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
        ]), num_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('ohe', OneHotEncoder(handle_unknown='ignore')),
        ]), cat_features),
    ]
)

X_train_lgb = prep_no_scale.fit_transform(X_train)
X_test_lgb = prep_no_scale.transform(X_test)

if sparse.issparse(X_train_lgb):
    X_train_lgb = X_train_lgb.tocsr()
    X_test_lgb = X_test_lgb.tocsr()

lgb_train = lgb.Dataset(X_train_lgb, label=y_train)

# custom AP evaluator for CV

def lgb_avg_precision(preds, data):
    y_true = data.get_label()
    return 'avg_precision', average_precision_score(y_true, preds), True

params = {
    'objective': 'binary',
    'metric': 'None',
    'verbosity': -1,
    'seed': SEED,
    'feature_pre_filter': False,
}

tuner = optuna_lgb.LightGBMTunerCV(
    params=params,
    train_set=lgb_train,
    num_boost_round=2000,
    folds=StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED),
    feval=lgb_avg_precision,
    early_stopping_rounds=100,
    optuna_seed=SEED,
    time_budget=900,  # 15 min budget; adjust in Colab if needed
    return_cvbooster=True,
    verbose_eval=False,
)

tuner.run()
best_params = tuner.best_params
print('Best params from LightGBMTunerCV:', best_params)

# Train final model using tuned params
best_num_boost_round = tuner.best_iteration
final_lgb = lgb.train(
    params=best_params,
    train_set=lgb_train,
    num_boost_round=best_num_boost_round,
)

lgb_test_proba = final_lgb.predict(X_test_lgb)
print('Tuned LGBM test AP:', average_precision_score(y_test, lgb_test_proba))
print('Tuned LGBM test ROC-AUC:', roc_auc_score(y_test, lgb_test_proba))

**Note:** Some LightGBM/Optuna integration versions prioritize built-in metrics; here we pass a custom `feval` for AP and still validate AP externally on the holdout set for final model selection.

## 6 — Calibration + thresholding (operational)

In [ ]:
# Select final model by highest test AP among tuned BRF and tuned LGBM
candidate_scores = {
    'BRF_tuned': brf_test_proba,
    'LGBM_tuned': lgb_test_proba,
}
final_name = max(candidate_scores, key=lambda n: average_precision_score(y_test, candidate_scores[n]))
final_scores = candidate_scores[final_name]

print('Selected final model by AP:', final_name)
print('Final AP:', average_precision_score(y_test, final_scores))
print('Final ROC-AUC:', roc_auc_score(y_test, final_scores))

for k in [0.01, 0.05, 0.10]:
    out = precision_recall_at_k(y_test.values, final_scores, k)
    print(out)


def threshold_for_targeting_rate(scores, targeting_rate):
    k = max(1, int(np.floor(len(scores) * targeting_rate)))
    return float(np.sort(scores)[-k])

example_rate = 0.05
thr = threshold_for_targeting_rate(final_scores, example_rate)
metrics_at_5 = precision_recall_at_k(y_test.values, final_scores, example_rate)
print(f"\nAt top {example_rate:.0%} targeting, threshold={thr:.5f}")
print(metrics_at_5)

In [ ]:
# Optional calibration check
prob_true, prob_pred = calibration_curve(y_test, final_scores, n_bins=10, strategy='quantile')
brier = brier_score_loss(y_test, final_scores)

plt.figure(figsize=(6,5))
plt.plot(prob_pred, prob_true, marker='o', label='Model')
plt.plot([0,1],[0,1],'k--', label='Perfectly calibrated')
plt.xlabel('Predicted probability')
plt.ylabel('Observed frequency')
plt.title(f'Reliability Curve (Brier={brier:.4f})')
plt.legend()
plt.tight_layout()
plt.show()

## 7 — Interpretability

In [ ]:
# Logistic regression coefficient plot (top +/-)
logit = models['LogisticRegression']
logit.fit(X_train, y_train)
feature_names = logit.named_steps['prep'].get_feature_names_out()
coefs = logit.named_steps['model'].coef_.ravel()
coef_df = pd.DataFrame({'feature': feature_names, 'coef': coefs})

top_pos = coef_df.nlargest(10, 'coef')
top_neg = coef_df.nsmallest(10, 'coef')
plot_df = pd.concat([top_neg, top_pos], axis=0)

plt.figure(figsize=(10, 7))
colors = ['#C44E52' if c < 0 else '#55A868' for c in plot_df['coef']]
plt.barh(plot_df['feature'], plot_df['coef'], color=colors)
plt.title('Logistic Regression: Top Positive/Negative Coefficients')
plt.xlabel('Coefficient value')
plt.tight_layout()
plt.show()

In [ ]:
# SHAP for tuned LightGBM model
# For speed/readability, use a sample
sample_n = min(2000, X_test_lgb.shape[0])
idx = np.random.choice(X_test_lgb.shape[0], size=sample_n, replace=False)
X_shap = X_test_lgb[idx]

explainer = shap.TreeExplainer(final_lgb)
shap_values = explainer.shap_values(X_shap)

# binary classifiers may return list [class0, class1]
if isinstance(shap_values, list):
    shap_values_to_plot = shap_values[1]
else:
    shap_values_to_plot = shap_values

feature_names_lgb = prep_no_scale.get_feature_names_out()
shap.summary_plot(shap_values_to_plot, X_shap, feature_names=feature_names_lgb, show=True)

**Business interpretation (example):** prioritize users with feature patterns associated with higher model scores (from SHAP/coefs). Use top-K targeting (e.g., top 5%) to control campaign volume while maximizing conversion lift. This notebook does **not** estimate revenue/profit.

## 8 — Save artifacts + inference example

In [ ]:
# Save artifacts
joblib.dump(preprocessor, 'preprocessor_scaled.joblib')
joblib.dump(prep_no_scale, 'preprocessor_lgb.joblib')
joblib.dump(best_brf, 'balanced_random_forest_tuned.joblib')
final_lgb.save_model('lightgbm_tuned.txt')

print('Artifacts saved.')

In [ ]:
# Inference example: top-k highest propensity users on the holdout test set
# Build an ID-aligned test frame
_, test_idx = train_test_split(
    work_df.index,
    test_size=0.2,
    stratify=work_df[TARGET],
    random_state=SEED,
)

id_test = work_df.loc[test_idx, [ID_COL]].copy().reset_index(drop=True)
score_df = id_test.copy()
score_df['propensity_score'] = final_scores
score_df = score_df.sort_values('propensity_score', ascending=False).reset_index(drop=True)

k_show = 20
display(score_df.head(k_show))